In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.spatial import Voronoi, voronoi_plot_2d
from vertexmodelpack import connections as fc
from vertexmodelpack import sppvertex as sppv
import importlib
from vertexmodelpack import readTissueFiles as rTF

In [ ]:
importlib.reload(sppv)

## Prior to tissue generation

In [1]:
def hex_big(Nw):
    n = 1
    Hexn = 1
    while Hexn <= Nw:
        Hexn = Hexn + 6*n
        n = n + 1
    # print(Hexn,n)
    return Hexn, n

In [ ]:
Nw = 10
epsilon = 0.003

N1 = int(np.ceil(np.sqrt(np.ceil(4/3*(1/epsilon+Nw))))**2)

Hx, n = hex_big(Nw)
N2 = int(np.ceil(np.sqrt(np.ceil(4/3*(Hx+30*(n+2)))))**2)

N = max(N1,N2)

In [ ]:
print(N, Hx)

# Loading tissue

In [2]:
size_of_wound = 0 #or the number of removed cells

In [ ]:
dataset = rTF.open_tissuefile('tissues/tissue405/size'+str(size_of_wound),0)

coords = dataset['centers']
vorPointRegion1 = dataset['point regions']
vorRegions = dataset['regions']
                
N = len(coords[:,0])
print(N)

vertices = dataset['vertices']

vorRidges = dataset['Edge connections']

wloc = dataset['WoundLoc']

vorPointRegion= []
for i in range(N):
    vorPointRegion.append(i)

Boundaries = dataset['boundaries']

vor = Voronoi(coords)
vor.point_region = vorPointRegion
vor.regions = vorRegions
vor.ridge_vertices = vorRidges
vor.vertices = vertices

# Visualization

## Visualize centers

In [ ]:
fig = plt.figure(figsize=(8,8))
ax = fig.add_subplot(111)
voronoi_plot_2d(vor,show_vertices=False,ax=ax,show_points=False)
plt.scatter(coords[:,0], coords[:,1],marker='s',linewidths=5)
for i, txt in enumerate(vorPointRegion):
    plt.annotate(int(txt),(coords[i,0], coords[i,1]),fontsize=13)
    
plt.plot(coords[wloc,0],coords[wloc,1],'ro',ms=10)
plt.xlim(-3,3)
plt.ylim(-3,3)

## Visualize vertices

In [ ]:
fig = plt.figure(figsize=(8,8))
ax = fig.add_subplot(111)
voronoi_plot_2d(vor,show_vertices=False,ax=ax,show_points=False)
plt.scatter(vertices[:,0], vertices[:,1],marker='s',linewidths=3)
for i, txt in enumerate(np.arange(0,len(vertices))):
    plt.annotate(int(txt),(vertices[i,0], vertices[i,1]),fontsize=20)
    
plt.plot(coords[wloc,0],coords[wloc,1],'ro',ms=10)
plt.ylim(-2.75,2.75)
plt.grid()

### Find 2-degree vertices

In [164]:
wound_vertices = vorRegions[wloc]
loosey = []
for v in wound_vertices:
    if len(fc.find_vertex_neighbour_vertices(vorRidges,v))<3:
        loosey.append(v)
cell_loosey = []
for v in loosey:
    print(fc.find_vertex_neighbour_vertices(vorRidges,v))
    nv = fc.find_vertex_neighbour_centers(vorRegions,vorPointRegion,v)[0]
    for c in nv:
        if c != wloc:
            if c not in cell_loosey:
                cell_loosey.append(c)

In [165]:
loc_loosey_edges = []
loc_loosey_edges_ind = []
for ind_e in range(len(vorRidges)):
    edge = vorRidges[ind_e]
    timesx = 0
    for elem in edge:
        if elem in loosey:
            timesx+=1
    if timesx!=0:
        loc_loosey_edges.append(ind_e)
        loc_loosey_edges_ind.append(timesx)
    

In [166]:
sorted_edges = np.argsort(loc_loosey_edges)

In [ ]:
print(np.sort(cell_loosey))
print(np.sort(loosey))
print(np.array(loc_loosey_edges)[sorted_edges])
print(np.array(loc_loosey_edges_ind)[sorted_edges])

In [173]:
nsides_cells = []
for a in vorPointRegion:
    nsides_cells.append(sppv.nsides_vor(vorPointRegion,vorRegions,a))

In [ ]:
fig = plt.figure(figsize=(16,4))
ax1 = fig.add_subplot(131)
voronoi_plot_2d(vor,show_vertices=False,ax=ax1,show_points=False)
plt.scatter(coords[vorPointRegion,0],coords[vorPointRegion,1], c=nsides_cells,s=150,cmap='rainbow')
plt.clim(3,8)
plt.colorbar()

In [ ]:
plt.plot(nsides_cells,'.')

## Visualize Boundaries

In [178]:
lbound_up = np.where(vertices[Boundaries[0],1]>3)
bound_up = np.array(Boundaries[0])[lbound_up]

In [ ]:

fig = plt.figure(figsize=(16,16))
ax = fig.add_subplot(111)
voronoi_plot_2d(vor,show_vertices=True,ax=ax)
ax.plot(vertices[Boundaries[1],0],vertices[Boundaries[1],1],'ro')
ax.plot(vertices[np.array(Boundaries[0]),0],vertices[np.array(Boundaries[0]),1],'go')
ax.plot(vertices[bound_up,0],vertices[bound_up,1],'ko')
for i, txt in enumerate(np.arange(0,len(vertices))):
    plt.annotate(int(txt),(vertices[i,0], vertices[i,1]),fontsize=12)
ax.plot(np.arange(-5,5.5), np.arange(-5,5.5)*0 + 3)
ax.plot(np.arange(-5,5.5), np.arange(-5,5.5)*0 - 3)
ax.plot(np.arange(-5,5)*0-4, np.arange(-5,5))
ax.plot(np.arange(-5,5)*0+4, np.arange(-5,5))  
ax.grid()
plt.show()

# Test rearrangement of wound area

In [ ]:
areaWound0 = sppv.area_vor(vorPointRegion,vorRegions,vertices,vorRidges,wloc,True)

In [183]:
areaBound0 = sppv.area_vor([0],Boundaries,vertices,vorRidges,0,True)

In [193]:
ordered_boundary =[1, 2, 3, 4, 0, 1]

In [ ]:
for v in range(len(ordered_boundary)):
    print(v,ordered_boundary[v],vertices[list(np.array(Boundaries[1])[ordered_boundary]),0][v])

In [72]:
ord_bd = [28, 45, 9, 46, 24, 48, 10, 47, 41, 7, 42, 49, 11, 50, 63, 20, 56, 14, 55, 51, 12, 52, 54, 13, 53, 19, 33, 2, 32, 73, 30, 1, 31, 25, 69, 74, 62, 18, 61, 38, 5, 39, 40, 6, 36, 4, 37, 15, 57, 65, 21, 64, 43, 8, 44, 60, 17, 59, 58, 16, 35, 3, 34, 22, 66, 67, 23, 68, 26, 70, 75, 72, 27, 71, 29, 0]

In [ ]:
plt.figure(figsize=(6,6))
plt.plot(vertices[list(np.array(Boundaries[1])[ordered_boundary]),0],vertices[list(np.array(Boundaries[1])[ordered_boundary]),1],'ro-')
plt.grid()

In [ ]:
fig = voronoi_plot_2d(vor,show_vertices=False, show_points = False)
fig.figimage
wound_poly =  [vor.vertices[i] for i in list(np.array(Boundaries[1])[ordered_boundary]) if fc.norm(vor.vertices[i]) < np.sqrt(4)*5]       
for region in vorRegions:
    if not -1 in region:
        polygon = [vertices[j] for j in region if fc.norm(vor.vertices[j]-vor.vertices[wloc]) < np.sqrt(350)]
        if len(polygon) <= 3:
            plt.fill(*zip(*polygon),'darkviolet',alpha=0.5)
        if len(polygon) == 4:
            plt.fill(*zip(*polygon),'midnightblue',alpha=0.5)
        if len(polygon) == 5:
            plt.fill(*zip(*polygon),'deepskyblue',alpha=0.5)
        if len(polygon) == 6:
            plt.fill(*zip(*polygon),'forestgreen',alpha=0.5)
        if (len(polygon) >= 7) and (len(polygon) < 11):
        	plt.fill(*zip(*polygon),'darkorange',alpha=0.5)
        
plt.plot(vertices[list(np.array(Boundaries[1])[ordered_boundary]),0],vertices[list(np.array(Boundaries[1])[ordered_boundary]),1],'r-',lw = 2)
ax = plt.gca()
ax.set_xticks([])
ax.set_yticks([])       
plt.show()

